Requires movingpandas, geopandas, holoviz, datetime

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/bamacgabhann/PublicTransportTracking/fyp?labpath=ptt.ipynb)

In [1]:
import geopandas as gpd
import movingpandas as mpd
import hvplot.pandas
from datetime import datetime, timedelta

import shapely as shp
import matplotlib.pyplot as plt

from shapely.geometry import Point, LineString, Polygon
from holoviews import opts

import warnings

warnings.filterwarnings("ignore")

plot_defaults = {"linewidth": 5, "capstyle": "round", "figsize": (10, 6), "legend": True}
opts.defaults(
    opts.Overlay(active_tools=["wheel_zoom"], frame_width=800, frame_height=600)
)

## INPUTS: Set filename/path for GPX and bus stop files to be used in this analysis, and the journey route/date/time for the plot title

In [16]:
gpx = "GPX/2023/304 to UL 2023-03-09 0811.gpx"
bus_stops = gpd.read_file("GPKG/stops_304_to_ul.gpkg")
journey_plot_title = "304 to UL 2023-03-09 08:11"

-Read GPX file to geopandas dataframe called 'gdf'

-Drop unnecessary columns

-Convert to movingpandas trajectory called 'track'

-Add columns for distance in m, and speed in kph

In [3]:
gdf = gpd.read_file(gpx, layer="track_points").set_index("time")
gdf.drop(
    columns=[
        "magvar",
        "geoidheight",
        "name",
        "cmt",
        "desc",
        "src",
        "link1_href",
        "link1_text",
        "link1_type",
        "link2_href",
        "link2_text",
        "link2_type",
        "sym",
        "type",
        "fix",
        "sat",
        "hdop",
        "vdop",
        "pdop",
        "ageofdgpsdata",
        "dgpsid",
    ],
    inplace=True,
)
track = mpd.Trajectory(gdf, 1)
track.add_speed(name="speed (km/h)", units=("km", "h"))
track.add_distance(units="m")

track.df

,track_fid,track_seg_id,track_seg_point_id,ele,geometry,traj_id,speed (km/h),distance
time,,,,,,,,
2023-03-09 08:11:41,0,0,0,4.0,POINT (-8.65166 52.64003),1,3.241699,0.000000
2023-03-09 08:11:42,0,0,1,4.0,POINT (-8.65166 52.64002),1,3.241699,0.900472
2023-03-09 08:11:43,0,0,2,4.0,POINT (-8.65167 52.64002),1,1.406724,0.390757
2023-03-09 08:11:44,0,0,3,4.0,POINT (-8.65167 52.64002),1,1.458313,0.405087
2023-03-09 08:11:45,0,0,4,4.0,POINT (-8.65168 52.64001),1,1.667321,0.463145
...,...,...,...,...,...,...,...,...
2023-03-09 09:32:29,0,0,4783,15.3,POINT (-8.57087 52.67272),1,0.000000,0.000000
2023-03-09 09:32:31,0,0,4784,15.3,POINT (-8.57087 52.67272),1,0.000000,0.000000
2023-03-09 09:32:32,0,0,4785,15.3,POINT (-8.57087 52.67272),1,0.000000,0.000000


## Minimum time stopped to count, in seconds

In [4]:
stopped_time = 15

Identify stationary periods of this duration or longer within a 30m diameter circle

In [5]:
detector = mpd.TrajectoryStopDetector(track)
stationary_points = detector.get_stop_points(
    min_duration=timedelta(seconds=stopped_time), max_diameter=30
)

Plot journey, bus stops, and stationary periods

In [13]:
journey_plot = track.hvplot(
    c="speed (km/h)",
    clim=(0, 60),
    line_width=7.0,
    x="Longitude",
    y="Latitude",
    xlabel="Longitude",
    ylabel="Latitude",
    title=journey_plot_title,
    clabel="Speed (km/h)",
    tiles="CartoLight",
    cmap="RdYlGn",
    colorbar=True,
)

stationary_plot = stationary_points.hvplot(
    geo=True, size="duration_s", color="deeppink"
)

bus_stops_plot = bus_stops.hvplot(
    geo=True, size=40, marker="+", color="blue"
)  

track_plot = (
    journey_plot * stationary_plot * bus_stops_plot
)

track_plot

:Overlay
   .Tiles.I    :Tiles   [x,y]
   .Path.I     :Path   [Longitude,Latitude]   (speed (km/h))
   .Points.I   :Points   [Longitude,Latitude]   (speed (km/h),triangle_angle)
   .Points.II  :Points   [Longitude,Latitude]   (duration_s)
   .Points.III :Points   [Longitude,Latitude]

Use the save icon to save the map image. The size can be adjusted in the top cell (adjust frame_width=800, frame_height=600)

List details of stationary periods of 15 seconds or more within a 30m diameter circle

In [7]:
stop_time_ranges = detector.get_stop_time_ranges(
    min_duration=timedelta(seconds=15), max_diameter=30
)
for x in stop_time_ranges:
    print(x)

Traj 1: 2023-03-09 08:11:41 - 2023-03-09 08:25:45 (duration: 0 days 00:14:04)
Traj 1: 2023-03-09 08:25:46 - 2023-03-09 08:33:45 (duration: 0 days 00:07:59)
Traj 1: 2023-03-09 08:34:20 - 2023-03-09 08:34:40 (duration: 0 days 00:00:20)
Traj 1: 2023-03-09 08:34:54 - 2023-03-09 08:37:38 (duration: 0 days 00:02:44)
Traj 1: 2023-03-09 08:38:07 - 2023-03-09 08:38:32 (duration: 0 days 00:00:25)
Traj 1: 2023-03-09 08:38:33 - 2023-03-09 08:39:00 (duration: 0 days 00:00:27)
Traj 1: 2023-03-09 08:39:01 - 2023-03-09 08:39:39 (duration: 0 days 00:00:38)
Traj 1: 2023-03-09 08:39:40 - 2023-03-09 08:39:55 (duration: 0 days 00:00:15)
Traj 1: 2023-03-09 08:39:56 - 2023-03-09 08:41:29 (duration: 0 days 00:01:33)
Traj 1: 2023-03-09 08:41:51 - 2023-03-09 08:42:25 (duration: 0 days 00:00:34)
Traj 1: 2023-03-09 08:43:14 - 2023-03-09 08:43:39 (duration: 0 days 00:00:25)
Traj 1: 2023-03-09 08:44:04 - 2023-03-09 08:44:34 (duration: 0 days 00:00:30)
Traj 1: 2023-03-09 08:44:48 - 2023-03-09 08:45:26 (duration: 0 d